# Import CSV

In [1]:


import pandas as pd
from datetime import datetime

df = pd.read_csv(".\\InputData\\Natural Gas Futures Historical Data 1997 - 2021.csv")

# print(df.head(10))

# Format Dates

In [2]:
from datetime import datetime

df["Date"] = df["Date"].apply(lambda x: datetime.strptime(x, "%b %d, %Y"))

# print(df.head(10))

# Convert Volumes to float

In [3]:
def convert_volume(vol_string):
    if vol_string == "-":
        return 0
    
    return 1000 * float(vol_string.replace("K", ""))

    
df["Vol."] = df["Vol."].apply(convert_volume)

# print(df.head(10))

# Drop Unneeded Columns

In [4]:
df = df.drop(columns=["Change %"])

print(df.head())

        Date  Price   Open   High    Low     Vol.
0 2021-04-23  50.72  52.15  52.55  49.40  25590.0
1 2021-04-22  53.86  53.25  54.71  53.00   7710.0
2 2021-04-21  52.82  52.82  53.29  52.20   5300.0
3 2021-04-20  52.82  52.00  53.64  51.52   9320.0
4 2021-04-19  52.48  52.00  53.10  51.85   6600.0
